# Create_Knowledge_Graph_using_LLM_Langchain.ipynb

#### Code concepts courtesy 

https://python.langchain.com/docs/use_cases/graph/

# Can LLM create and query knowledge graph

### Install libraries

In [ ]:
# list of packages

!pip install langchain==0.1.13
!pip install langchain-community==0.0.29
!pip install langchain-core==0.1.33
!pip install langchain-experimental==0.0.55
!pip install langchain-openai==0.1.1
!pip install langchain-text-splitters==0.0.1
!pip install langsmith==0.1.31
!pip install openai==1.14.2

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()


In [2]:

import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = os.getenv("neo4j_uri")
os.environ["NEO4J_USERNAME"] = os.getenv("neo4j_username")
os.environ["NEO4J_PASSWORD"] = os.getenv("neo4j_password")

graph = Neo4jGraph()

### Set up graph transformer

In [4]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI


graph_llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo-preview")

graph_llm_transformer = LLMGraphTransformer(llm=graph_llm)


### Create nodes and relationships

In [5]:
from langchain_core.documents import Document

text = """
Harry potter studies at hogwarts.
Harry potter is a friend of Ron Weasley. Ron Weasley studies at hogwarts.
Hermoine Granger is a friend of Ron Weasly.
"""
text_document = [Document(page_content=text)]
graph_text_document = graph_llm_transformer.convert_to_graph_documents(text_document)
print ('nodes')
for n in graph_text_document[0].nodes:
    print (n)
print ('relationships')
for r in graph_text_document[0].relationships:
    print (r)

nodes
id='Harry Potter' type='Person'
id='Hogwarts' type='Organization'
id='Ron Weasley' type='Person'
id='Hermoine Granger' type='Person'
relationships
source=Node(id='Harry Potter', type='Person') target=Node(id='Hogwarts', type='Organization') type='STUDIES_AT'
source=Node(id='Harry Potter', type='Person') target=Node(id='Ron Weasley', type='Person') type='FRIEND_OF'
source=Node(id='Ron Weasley', type='Person') target=Node(id='Hogwarts', type='Organization') type='STUDIES_AT'
source=Node(id='Hermoine Granger', type='Person') target=Node(id='Ron Weasley', type='Person') type='FRIEND_OF'


### Add to graph db

In [6]:
graph.add_graph_documents(graph_text_document)

### visualize nodes from console 

https://workspace-preview.neo4j.io/workspace/query

![org_nodes](org_nodes.png)

### But we want the node to be of type school 

In [27]:
import os

neo4j_uri = os.getenv("neo4j_uri")
neo4j_username = os.getenv("neo4j_username")
neo4j_password = os.getenv("neo4j_password")
from neo4j import GraphDatabase

neo4j_auth = (neo4j_username,neo4j_password)
# delete nodes for now

driver = GraphDatabase.driver(neo4j_uri, auth=neo4j_auth)

def delete_graph(tx):
    # Delete relationships first
    tx.run("MATCH (n)-[r]-() DELETE r")
    
    # Then delete nodes
    tx.run("MATCH (n) DELETE n")

# Execute the function within a Neo4j session
with driver.session() as session:
    session.write_transaction(delete_graph)

driver.close()  # Close the driver connection when done

/var/folders/86/th81x23971s_drqsdmgk1fz40000gp/T/ipykernel_12038/1862675062.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(delete_graph)


### Specify Custom allowed nodes for the transformer 

In [28]:
graph_llm_transformer = LLMGraphTransformer(llm=graph_llm, allowed_nodes=["Person", "School"],
                                           allowed_relationships=["STUDIES_AT", "FRIEND_OF"])
from langchain_core.documents import Document

text = """
Harry potter studies at hogwarts.
Harry potter is a friend of Ron Weasley. Ron Weasley studies at hogwarts.
Hermoine Granger is a friend of Ron Weasly.
"""
text_document = [Document(page_content=text)]
graph_text_document = graph_llm_transformer.convert_to_graph_documents(text_document)
print ('nodes')
for n in graph_text_document[0].nodes:
    print (n)
print ('relationships')
for r in graph_text_document[0].relationships:
    print (r)

nodes
id='Harry Potter' type='Person'
id='Hogwarts' type='School'
id='Ron Weasley' type='Person'
id='Hermoine Granger' type='Person'
relationships
source=Node(id='Harry Potter', type='Person') target=Node(id='Hogwarts', type='School') type='STUDIES_AT'
source=Node(id='Harry Potter', type='Person') target=Node(id='Ron Weasley', type='Person') type='FRIEND_OF'
source=Node(id='Ron Weasley', type='Person') target=Node(id='Hogwarts', type='School') type='STUDIES_AT'
source=Node(id='Hermoine Granger', type='Person') target=Node(id='Ron Weasley', type='Person') type='FRIEND_OF'


### Add to graph db

In [29]:
graph.add_graph_documents(graph_text_document)

Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
Failed to write data to connection IPv4Address(('67b33eac.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))


### visualize nodes from console
https://workspace-preview.neo4j.io/workspace/query

![custom_org_school](custom_org_school.png)

![custom_all_nodes](custom_all_nodes.png)

### Use LLM to query the graph DB

In [30]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

students_response = chain.invoke({"query": """which person studies at Hogwarts ?. For the properties always
use id instead of name"""})
students_response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:STUDIES_AT]->(s:School {id: "Hogwarts"}) RETURN p.id;
Full Context:
[{'p.id': 'Harry Potter'}, {'p.id': 'Ron Weasley'}]

> Finished chain.


{'query': 'which person studies at Hogwarts ?. For the properties always\nuse id instead of name',
 'result': 'Harry Potter and Ron Weasley study at Hogwarts.'}

In [53]:
internal_knowledge = students_response['result']
internal_knowledge

'Harry Potter and Ron Weasley study at Hogwarts.'

### Use case query LLM with out internal data from knowledge graph

In [41]:
question = """name of the students who study at Hogwarts """

In [45]:
import langchain.prompts.chat
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that answers users questions."),
    HumanMessage(content=question),
]
llm_response = llm.invoke(messages)
print ('llm_response',llm_response.content)

llm_response Hogwarts School of Witchcraft and Wizardry, the fictional school from J.K. Rowling's "Harry Potter" series, has a vast number of students across its history. Here are names of some of the most notable students from the series, categorized by their respective houses:

### Gryffindor
- Harry Potter
- Hermione Granger
- Ron Weasley
- Neville Longbottom
- Ginny Weasley
- Fred Weasley
- George Weasley
- Seamus Finnigan
- Dean Thomas
- Parvati Patil
- Lavender Brown
- Colin Creevey
- Lee Jordan

### Hufflepuff
- Cedric Diggory
- Nymphadora Tonks (though her time at Hogwarts is not detailed in the series, it's known she was in Hufflepuff)
- Ernie Macmillan
- Hannah Abbott
- Justin Finch-Fletchley
- Susan Bones

### Ravenclaw
- Luna Lovegood
- Cho Chang
- Padma Patil
- Terry Boot
- Michael Corner
- Anthony Goldstein

### Slytherin
- Draco Malfoy
- Vincent Crabbe
- Gregory Goyle
- Pansy Parkinson
- Blaise Zabini
- Millicent Bulstrode
- Theodore Nott

This list includes only a fract

### Retreieval Augmented Generation - query LLM with internal data from knowledge graph

In [52]:
internal_knowledge = students_response['result']
internal_knowledge

'Harry Potter and Ron Weasley study at Hogwarts.'

In [54]:
question

'name of the students who study at Hogwarts '

In [51]:


system_message_context = f"""You are a helpful assistant that answers users questions .
    Only use the internal_knowledge provided, do not add any additional information.
    internal_knowledge:  {internal_knowledge}
    """

messages = [
    SystemMessage(content= system_message_context),
    HumanMessage(content=question),
]
llm_response_graph_knowledge = llm.invoke(messages)
print ('llm_response_graph_knowledge\n',llm_response_graph_knowledge.content)

llm_response_graph_knowledge
 Harry Potter and Ron Weasley study at Hogwarts.


### Can LLM do advanced query 

In [23]:
response = chain.invoke({"query": """who might Hermoine Granger become friends with?.
For the properties always use id instead of name. For example instead of p.name use p.id"""})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p1 {id: "Hermoine Granger"})-[:KNOWS]->(p2)-[:KNOWS]->(p3)
WHERE NOT (p1)-[:KNOWS]->(p3)
RETURN p3.id AS PotentialFriend

Full Context:
[]

> Finished chain.


{'query': 'who might Hermoine Granger become friends with?.\nFor the properties always use id instead of name. For example instead of p.name use p.id',
 'result': "I don't know the answer."}

### Looks like it needs more prompting